In [114]:
import logging
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-114-f0b900f0fc61>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [115]:
db_conn = db.get_db_conn()

In [120]:
db_conn.close()

## Passing Summary

In [116]:
## total_epa and rush_epa don't have qb_kneels in them
query = """
    SELECT
        p.*,
        scrambles.attempts as scramble_attempts,
        scrambles.yards as scramble_yards,
        scrambles.td as scramble_td,
        scrambles.fumbles_lost as scramble_fumbles_lost,
        scrambles.epa as scramble_epa,
        
        designed.attempts as designed_rush_attempts,
        designed.yards as designed_rush_yards,
        designed.td as designed_rush_td,
        designed.fumbles_lost as designed_fumbles_lost,
        designed.epa as designed_rush_epa,

        qb_kneels.attempts as qb_kneel_attempts,
        qb_kneels.yards as qb_kneel_yards,
        qb_kneels.epa as qb_kneel_epa,
        
        total_rush.attempts as total_rush_attempts,
        total_rush.yards as total_rush_yards,
        total_rush.td as total_rush_td,
        total_rush.fumbles_lost as total_rush_fumbles_lost,
        total_rush.epa as total_rush_epa,

        receiving.receptions as receptions,
        receiving.targets as receiving_targets,
        receiving.yards as receiving_yards,
        receiving.td as receiving_td,
        receiving.air_yards_intended as air_yards,
        receiving.fumbles_lost as receiving_fumbles_lost,
        receiving.epa as receiving_epa,
        
        scrambles.attempts + designed.attempts AS rush_attempts,
        scrambles.yards + designed.yards AS rush_yards,
        scrambles.td + designed.td AS rush_td,
        scrambles.epa + designed.epa AS rush_epa,
        total_rush.fumbles_lost + p.fumbles_lost AS total_fumbles_lost,
        scrambles.epa + designed.epa + 
            (CASE WHEN receiving.epa IS NULL THEN 0 ELSE receiving.epa END) + p.epa AS total_epa
    
    FROM passing_by_player_by_game p
    
    LEFT JOIN rushing_by_player_by_game scrambles
    ON 
        p.gsis_id = scrambles.gsis_id
        and p.game_id = scrambles.game_id
        and p.week = scrambles.week
        and p.team = scrambles.team
        and scrambles.rush_type = 'scramble'
        
    LEFT JOIN rushing_by_player_by_game designed
    ON 
        p.gsis_id = designed.gsis_id
        and p.game_id = designed.game_id
        and p.week = designed.week
        and p.team = designed.team
        and designed.rush_type = 'designed'

    LEFT JOIN rushing_by_player_by_game qb_kneels
    ON 
        p.gsis_id = qb_kneels.gsis_id
        and p.game_id = qb_kneels.game_id
        and p.week = qb_kneels.week
        and p.team = qb_kneels.team
        and qb_kneels.rush_type = 'qb_kneel'

    LEFT JOIN rushing_by_player_by_game total_rush
    ON 
        p.gsis_id = total_rush.gsis_id
        and p.game_id = total_rush.game_id
        and p.week = total_rush.week
        and p.team = total_rush.team
        and total_rush.rush_type = 'total'
        
    LEFT JOIN receiving_by_player_by_game receiving
    ON 
        p.gsis_id = receiving.gsis_id
        and p.game_id = receiving.game_id
        and p.week = receiving.week
        and p.team = receiving.team
"""
df = pd.read_sql(query, db_conn)
df = df.fillna(0)
df.sort_values('total_epa', ascending=False).head(20)

,year,season_type,game_id,team,opp,week,gsis_id,pos,passer,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,sacks,sack_yards,fumbles,fumbles_lost,spikes,pass_epa,spike_epa,epa,cpoe,scramble_attempts,scramble_yards,scramble_td,scramble_fumbles_lost,scramble_epa,designed_rush_attempts,designed_rush_yards,designed_rush_td,designed_fumbles_lost,designed_rush_epa,qb_kneel_attempts,qb_kneel_yards,qb_kneel_epa,total_rush_attempts,total_rush_yards,total_rush_td,total_rush_fumbles_lost,total_rush_epa,receptions,receiving_targets,receiving_yards,receiving_td,air_yards,receiving_fumbles_lost,receiving_epa,rush_attempts,rush_yards,rush_td,rush_epa,total_fumbles_lost,total_epa
516,2020,REG,2020_15_BUF_DEN,BUF,DEN,15,00-0034857,QB,J.Allen,28,41,359,404,241,2,0,1,-7,1,0,0,25.663771,0.000000,25.663771,11.202372,2.0,25.0,2.0,0.0,4.172348,1.0,8.0,0.0,0.0,0.496162,0.0,0.0,0.000000,3.0,33.0,2.0,0.0,4.668510,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,33.0,2.0,4.668510,0.0,30.332281
109,2020,REG,2020_06_ATL_MIN,ATL,MIN,6,00-0026143,QB,M.Ryan,30,42,371,270,200,4,0,2,-8,0,0,0,30.441585,0.000000,30.441585,8.848381,2.0,13.0,0.0,0.0,-0.178477,1.0,-4.0,0.0,0.0,-0.992540,1.0,-1.0,-1.700265,4.0,8.0,0.0,0.0,-2.871282,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,9.0,0.0,-1.171017,0.0,29.270568
248,2020,REG,2020_15_DET_TEN,TEN,DET,15,00-0029701,QB,R.Tannehill,21,28,273,246,142,3,0,1,-5,0,0,0,23.807681,0.000000,23.807681,14.321147,1.0,1.0,0.0,0.0,-0.586671,2.0,20.0,2.0,0.0,3.074767,0.0,0.0,0.000000,3.0,21.0,2.0,0.0,2.488096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,21.0,2.0,2.488096,0.0,26.295777
504,2020,REG,2020_02_BUF_MIA,BUF,MIA,2,00-0034857,QB,J.Allen,24,36,415,362,261,4,0,1,-4,0,0,0,23.438254,0.000000,23.438254,6.101214,1.0,4.0,0.0,0.0,-0.527615,1.0,16.0,0.0,0.0,1.156817,2.0,-1.0,-1.414212,4.0,19.0,0.0,0.0,-0.785011,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.0,20.0,0.0,0.629201,0.0,24.067456
404,2020,REG,2020_06_HOU_TEN,HOU,TEN,6,00-0033537,QB,D.Watson,28,39,335,327,223,4,0,2,-15,0,0,1,21.481806,0.345304,21.827110,11.588879,3.0,18.0,0.0,0.0,1.022265,1.0,8.0,0.0,0.0,1.134510,0.0,0.0,0.000000,4.0,26.0,0.0,0.0,2.156775,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,26.0,0.0,2.156775,0.0,23.983885
517,2020,REG,2020_16_BUF_NE,BUF,NE,16,00-0034857,QB,J.Allen,27,36,320,359,183,4,0,0,0,0,0,0,18.374526,0.000000,18.374526,16.230555,1.0,11.0,0.0,0.0,1.606148,2.0,25.0,0.0,0.0,3.573005,1.0,-1.0,0.000000,4.0,35.0,0.0,0.0,5.179153,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,36.0,0.0,5.179153,0.0,23.553679
437,2020,REG,2020_12_KC_TB,KC,TB,12,00-0033873,QB,P.Mahomes,37,51,462,485,306,3,0,2,-6,1,1,0,21.338562,0.000000,21.338562,15.876680,2.0,12.0,0.0,0.0,0.781673,1.0,17.0,0.0,0.0,1.715619,1.0,-1.0,-1.514402,4.0,28.0,0.0,0.0,0.982890,0.0,1.0,0.0,0.0,1.0,0.0,-0.425582,3.0,29.0,0.0,2.497292,1.0,23.410271
9,2020,REG,2020_10_TB_CAR,TB,CAR,10,00-0019596,QB,T.Brady,28,40,341,324,173,3,0,1,-7,0,0,0,23.463633,0.000000,23.463633,12.335993,1.0,1.0,0.0,0.0,-1.138555,1.0,1.0,1.0,0.0,0.730949,0.0,0.0,0.000000,2.0,2.0,1.0,0.0,-0.407605,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2.0,2.0,1.0,-0.407605,0.0,23.056028
649,2020,REG,2020_15_LAC_LV,LAC,LV,15,00-0036355,QB,J.Herbert,22,33,314,359,223,2,0,1,-8,0,0,0,22.069279,0.000000,22.069279,5.485422,1.0,1.0,0.0,0.0,-0.787411,3.0,13.0,1.0,0.0,1.619520,0.0,0.0,0.000000,4.0,14.0,1.0,0.0,0.832109,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,14.0,1.0,0.832109,0.0,22.901388
351,2020,REG,2020_02_LA_PHI,LA,PHI,2,00-0033106,QB,J.Goff,20,28,267,221,158,3,0,1,-9,0,0,0,21.196814,0.000000,21.196814,15.070518,2.0,11.0,0.0,0.0,-0.170811,2.0,5.0,0.0,0.0,1.666722,3.0,-3.0,-2.277415,7.0,13.0,0.0,0.0,-0.781503,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.0,16.0,0.0,1.495911,0.0,22.692725


In [117]:
sum(df.isnull().sum(axis = 0))

0

In [118]:
def get_fantasy_points(pass_tds: int,
                       ints: int,
                       pass_yards: int,
                       rush_yards: int,
                       rush_tds: int,
                       receptions: int,
                       rec_yards: int,
                       rec_tds: int,
                       fumbles_lost: int,
                       ppr: float = 0.5) -> float:
    return (pass_tds * 4) \
        + (pass_yards / 25) \
        - (ints * 2) \
        + (rush_yards / 10) \
        + (rush_tds * 6) \
        + (receptions * ppr) \
        + (rec_yards / 10) \
        + (rec_tds * 6) \
        - (fumbles_lost * 2)


In [119]:
def _transform(df: pd.DataFrame) -> pd.DataFrame:
    df['fantasy_points_half'] = df.apply(
        lambda row: get_fantasy_points(
            row['td'], row['int'], row['yards'],
            row['total_rush_yards'], row['total_rush_td'],
            row['receptions'], row['receiving_yards'],
            row['receiving_td'], row['total_fumbles_lost']
        ), axis=1
    )
    return df
    
df = _transform(df)
df.sort_values('fantasy_points_half', ascending=False).head()

,year,season_type,game_id,team,opp,week,gsis_id,pos,passer,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,sacks,sack_yards,fumbles,fumbles_lost,spikes,pass_epa,spike_epa,epa,cpoe,scramble_attempts,scramble_yards,scramble_td,scramble_fumbles_lost,scramble_epa,designed_rush_attempts,designed_rush_yards,designed_rush_td,designed_fumbles_lost,designed_rush_epa,qb_kneel_attempts,qb_kneel_yards,qb_kneel_epa,total_rush_attempts,total_rush_yards,total_rush_td,total_rush_fumbles_lost,total_rush_epa,receptions,receiving_targets,receiving_yards,receiving_td,air_yards,receiving_fumbles_lost,receiving_epa,rush_attempts,rush_yards,rush_td,rush_epa,total_fumbles_lost,total_epa,fantasy_points_half
429,2020,REG,2020_03_KC_BAL,KC,BAL,3,00-0033873,QB,P.Mahomes,31,42,385,346,207,4,0,0,0,0,0,0,31.377849,0.000000,31.377849,9.176690,4.0,26.0,1.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,4.0,26.0,1.0,0.0,3.874581,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,40.00
346,2020,REG,2020_02_ATL_DAL,DAL,ATL,2,00-0033077,QB,D.Prescott,34,48,450,371,245,1,0,1,-5,1,1,1,12.162108,0.352055,12.514162,4.149191,1.0,11.0,0.0,0.0,0.057622,4.0,7.0,3.0,0.0,2.802679,0.0,0.0,0.000000,5.0,18.0,3.0,0.0,2.860301,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.0,18.0,3.0,2.860301,1.0,15.374463,39.80
641,2020,REG,2020_07_JAX_LAC,LAC,JAX,7,00-0036355,QB,J.Herbert,27,44,347,383,152,3,0,1,-2,0,0,0,14.643816,0.000000,14.643816,-3.253459,3.0,31.0,1.0,0.0,3.860812,2.0,38.0,0.0,0.0,2.575262,4.0,-3.0,-3.562504,9.0,66.0,1.0,0.0,2.873569,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.0,69.0,1.0,6.436073,0.0,21.079890,38.48
171,2020,REG,2020_17_NYJ_NE,NE,NYJ,17,00-0027939,QB,C.Newton,21,33,242,182,114,3,0,3,-23,0,0,0,10.564337,0.000000,10.564337,1.014256,0.0,0.0,0.0,0.0,0.000000,11.0,79.0,0.0,0.0,2.036486,0.0,0.0,0.000000,11.0,79.0,0.0,0.0,2.036486,1.0,1.0,19.0,1.0,13.0,0.0,2.112464,0.0,0.0,0.0,0.000000,0.0,0.000000,37.98
550,2020,REG,2020_09_MIA_ARI,ARI,MIA,9,00-0035228,QB,K.Murray,21,27,283,250,231,3,0,1,-19,1,1,0,9.647349,0.000000,9.647349,12.428653,5.0,33.0,0.0,0.0,1.277929,6.0,73.0,1.0,0.0,9.315312,0.0,0.0,0.000000,11.0,106.0,1.0,0.0,10.593242,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,11.0,106.0,1.0,10.593242,1.0,20.240591,37.92


## Trying to create a mega summary

In [ ]:
## total_epa and rush_epa don't have qb_kneels in them
query = """
    SELECT
        p.*,
        scrambles.attempts as scramble_attempts,
        scrambles.yards as scramble_yards,
        scrambles.td as scramble_td,
        scrambles.fumbles_lost as scramble_fumbles_lost,
        scrambles.epa as scramble_epa,
        
        designed.attempts as designed_rush_attempts,
        designed.yards as designed_rush_yards,
        designed.td as designed_rush_td,
        designed.fumbles_lost as designed_fumbles_lost,
        designed.epa as designed_rush_epa,

        qb_kneels.attempts as qb_kneel_attempts,
        qb_kneels.yards as qb_kneel_yards,
        qb_kneels.epa as qb_kneel_epa,
        
        total_rush.attempts as total_rush_attempts,
        total_rush.yards as total_rush_yards,
        total_rush.td as total_rush_td,
        total_rush.fumbles_lost as total_rush_fumbles_lost,
        total_rush.epa as total_rush_epa,

        receiving.receptions as receptions,
        receiving.targets as receiving_targets,
        receiving.yards as receiving_yards,
        receiving.td as receiving_td,
        receiving.air_yards_intended as air_yards,
        receiving.fumbles_lost as receiving_fumbles_lost,
        receiving.epa as receiving_epa,
        
        scrambles.attempts + designed.attempts AS rush_attempts,
        scrambles.yards + designed.yards AS rush_yards,
        scrambles.td + designed.td AS rush_td,
        scrambles.epa + designed.epa AS rush_epa,
        total_rush.fumbles_lost + p.fumbles_lost AS total_fumbles_lost,
        scrambles.epa + designed.epa + 
            (CASE WHEN receiving.epa IS NULL THEN 0 ELSE receiving.epa END) + p.epa AS total_epa
    
    FROM passing_by_player_by_game p
    
    JOIN rushing_by_player_by_game scrambles
    ON 
        p.gsis_id = scrambles.gsis_id
        and p.game_id = scrambles.game_id
        and p.week = scrambles.week
        and p.team = scrambles.team
        and scrambles.rush_type = 'scramble'
        
    LEFT JOIN rushing_by_player_by_game designed
    ON 
        p.gsis_id = designed.gsis_id
        and p.game_id = designed.game_id
        and p.week = designed.week
        and p.team = designed.team
        and designed.rush_type = 'designed'

    LEFT JOIN rushing_by_player_by_game qb_kneels
    ON 
        p.gsis_id = qb_kneels.gsis_id
        and p.game_id = qb_kneels.game_id
        and p.week = qb_kneels.week
        and p.team = qb_kneels.team
        and qb_kneels.rush_type = 'qb_kneel'

    LEFT JOIN rushing_by_player_by_game total_rush
    ON 
        p.gsis_id = total_rush.gsis_id
        and p.game_id = total_rush.game_id
        and p.week = total_rush.week
        and p.team = total_rush.team
        and total_rush.rush_type = 'total'
        
    LEFT JOIN receiving_by_player_by_game receiving
    ON 
        p.gsis_id = receiving.gsis_id
        and p.game_id = receiving.game_id
        and p.week = receiving.week
        and p.team = receiving.team
"""
df = pd.read_sql(query, db_conn)
df = df.fillna(0)
df.sort_values('total_epa', ascending=False).head(20)